In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.20

## 전동물건 처리

In [2]:
interface Vehicle {                                  // 인터페이스를 정의한다
    fun go(): String                                 // 추상 메소드 정의 
    fun display() = "움직이는 물건 인터페이스"             // 일반 메소드 정의 
}

class CarImpl(val where: String): Vehicle {          // 인터페이스 구현 클래스 
    override fun go() = "차로 어디까지 : $where"
    override fun display() = "자동차 클래스"
}

class AirplaneImpl(val where: String): Vehicle {     // 인터페이스 구현 클래스 
    override fun go() = "비행기로 어디까지 $where"
    override fun display() = "비행기 클래스"
}

class CarOrAirplane(val model: String,               // 위임을 작성한 클래스 
                    val impl: Vehicle): 
                               Vehicle by impl {     // 최상의 인터페이스에 위임처리
    fun tellTrip() {                                 // 위임을 호출할 메소드 작성
        println("$model ${this.go()}")   
    }
}

val m330 = CarOrAirplane("포르쉐 330",                 // 자동차  객체 생성 
                         CarImpl("Seoul"))
val m787 = CarOrAirplane("보잉  787",                  // 비행기 객체 생성 
                         AirplaneImpl("Seoul"))
    
m330.tellTrip()                                       // 객체의 메소드를 호출해서 위임 메소드 처리
println(m330.display())                               // 일반메소드 호출 
m787.tellTrip()             
println(m787.display())

포르쉐 330 차로 어디까지 : Seoul
자동차 클래스
보잉  787 비행기로 어디까지 Seoul
비행기 클래스


## 위임 클래스에 데이터 저장 및 확인

In [3]:
class CounterSetS<T>(                                             // 집합을 위임처리로 사용하는 클래스 
   private val innerSet: MutableSet<T> = mutableSetOf()
                                 ) : MutableSet<T> by innerSet {  // 집합을 위임 처리

   var elementsAdded: Int = 0                                     // 추가된 원소를 관리
       private set                                                // 속성은 내부 메소드에서만 갱신 

   override fun add(element: T): Boolean {                        // 원소 추가 
       elementsAdded++                                            // 속성값 변경 
       return innerSet.add(element)                               // 위임 메소드 호출 
   }

   override fun addAll(elements: Collection<T>): Boolean {        // 여러 원소를 추가 
       elementsAdded += elements.size                             // 속성 갱신 
       return innerSet.addAll(elements)                           // 위임 메소드 호출 
   }
   
   fun display() = innerSet                                       // 데이터 저장이 위임된 상태 확인
}

val counterList = CounterSetS<String>()                           // 객체를 생성 
counterList.addAll(listOf("A", "B", "C","D","E"))
println(counterList.elementsAdded)                                // 원소의 객체를 확인
println(counterList.display())                                    // 위임 클래스의 상태 확인 

5
[A, B, C, D, E]


## 계약과 잔액 관리를 분리

### 계좌와 잔액을 통합관리

In [4]:
class Account(val accno:Int,              // 하나의 계좌에서 계좌번호와 잔액관리 
              var balance : Int) {
    fun deposite(amount:Int) {            // 입금 메소드 
        balance = balance +amount
    }
    fun withdraw(amount:Int) {            // 출금 메소드 
        balance = balance - amount
    }
}

val acc = Account(1,0)                    // 계좌 생성 
acc.deposite(1000)                        // 입금
println(acc.balance)
acc.withdraw(1000)                        // 출금
println(acc.balance)

1000
0


### 위임을 통한 계좌와 잔액 분리

In [5]:
class Balance(val accno : Int,                          // 잔애관리 클래스 
              var balance : Int)

interface Accoutable {                                  // 입출금 처리 인터페이스 
    fun deposite(acc: Balance, amount:Int) 
    fun withdraw(acc: Balance, amount:Int) 
}

class DWManager : Accoutable {                          // 입출금을 구현한 클래스 
    override fun deposite(acc: Balance,                 // 입금 메소드 구현
                          amount:Int) { 
            acc.balance = acc.balance +amount
    }
    override fun withdraw(acc: Balance,                 // 출금 메소드 구현 
                          amount:Int) { 
        acc.balance = acc.balance - amount
    }
}

class Agreement(val accno : Int,                        // 계좌관리 클래스 
                val dwmanager : Accoutable) :           // 입출금관리 클래스
                         Accoutable by dwmanager 
                             
                         
val dwm = DWManager()                                   // 입출금관리 객체
val b = Balance(1,0)                                    // 잔액관리 객체
val a = Agreement(1,dwm)                                // 계약관리 객체 

a.deposite(b, 1000)                                     // 입금 실행 
println("계좌번호 :" + a.accno)
println("계좌번호 : ${b.accno} 잔액 : ${b.balance}")       // 잔액 확인 
a.withdraw(b, 500) 
println("계좌번호 : ${b.accno} 잔액 : ${b.balance}")       // 잔액 확인 

계좌번호 :1
계좌번호 : 1 잔액 : 1000
계좌번호 : 1 잔액 : 500


## Mixin 패턴 처리

In [6]:
class Balance (val accno: Int, var balande : Int) {        // 잔액을 관리하는 클래스
    override fun toString() =                              // 잔액을 출력 
             "Balance(accno=$accno, balance=$balande )"
}

interface Deposiable {                                     // 입금처리 인터페이스
    fun deposite(balance : Balance, 
                 amount : Int)
}

class Deposite  : Deposiable {                             // 입금 인터페이스를 구현한 클래스
    override fun deposite(balance : Balance, 
                          amount : Int) {
        balance.balande += amount
    }
}

interface Withdrawable {                                   // 출금처리 인터페이스  
    fun withdraw(balance : Balance, 
                amount : Int)
}

class Withdraw  : Withdrawable {                           // 출금 인터페이스를 구현한 클래스 
    override fun withdraw(balance : Balance, 
                          amount : Int) {
        balance.balande -= amount
    }
}

In [7]:
class Agreements(val accno : Int,                          // 위임처리 클래스 정의 
                 val with : Withdrawable,                  // 출금처리
                 val depo : Deposiable) :                  // 입금처리
                      Withdrawable by with,                // 출금입금 위임처리
                      Deposiable  by depo {
    
}

val ag = Agreements(1,Withdraw(), Deposite())              // 계약관리 객체 생성
val bal = Balance(1,0)                                     // 잔액관리 객체 생성 

ag.deposite(bal,10000)                                     // 입금처리
println(bal)
ag.withdraw(bal,9000)                                      // 출금처리
println(bal)

Balance(accno=1, balance=10000 )
Balance(accno=1, balance=1000 )
